In [324]:
import pandas as pd
import numpy as np

In [325]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [411]:
from sklearn.feature_selection import f_classif
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [326]:
csv=pd.read_csv('Downloads/HackUTD-2023-HomeBuyerInfo.csv')
df=pd.DataFrame(csv)

In [327]:
df.columns

Index(['ID', 'GrossMonthlyIncome', 'CreditCardPayment', 'CarPayment',
       'StudentLoanPayments', 'AppraisedValue', 'DownPayment', 'LoanAmount',
       'MonthlyMortgagePayment', 'CreditScore'],
      dtype='object')

In [328]:
df.head(10)

,ID,GrossMonthlyIncome,CreditCardPayment,CarPayment,StudentLoanPayments,AppraisedValue,DownPayment,LoanAmount,MonthlyMortgagePayment,CreditScore
0,1,3103.0,317.0,374.0,250.0,268468.0,32216.16,236251.84,1127.90,778
1,2,2939.0,368.0,395.0,228.0,335467.0,30192.03,305274.97,2240.00,546
2,3,4676.0,222.0,352.0,368.0,461177.0,36894.16,424282.84,2277.64,736
3,4,8469.0,224.0,416.0,400.0,326904.0,22883.28,304020.72,1451.44,761
4,5,2406.0,497.0,385.0,444.0,472661.0,23633.05,449027.95,2987.39,615
5,6,9128.0,219.0,424.0,220.0,403922.0,117137.38,286784.62,1719.42,661
6,7,8717.0,249.0,363.0,395.0,365483.0,73096.60,292386.40,1232.71,811
7,8,8194.0,315.0,353.0,303.0,269907.0,72874.89,197032.11,1310.86,648
8,9,8582.0,261.0,435.0,204.0,248121.0,12406.05,235714.95,1265.37,742
9,10,8832.0,351.0,470.0,203.0,408375.0,20418.75,387956.25,2846.69,554


### Credit rating >640
### Loan to value <80% (PMI requirement) {LTV = loan_amt/appraised_value} 
### Debt to income ratio : (CCP+MMP+SLP+CP)/(GMI) =k 
if k<0.36 : 'Y'//
if 0.36<=k<0.43 : 'Maybe'//
if k>0.43 : 'N'

### mortgage as a percentage of GMI: (MMP)/(GMI)<0.28

In [329]:
df[df['CreditScore']<640]

,ID,GrossMonthlyIncome,CreditCardPayment,CarPayment,StudentLoanPayments,AppraisedValue,DownPayment,LoanAmount,MonthlyMortgagePayment,CreditScore
1,2,2939.0,368.0,395.0,228.0,335467.0,30192.03,305274.97,2240.00,546
4,5,2406.0,497.0,385.0,444.0,472661.0,23633.05,449027.95,2987.39,615
9,10,8832.0,351.0,470.0,203.0,408375.0,20418.75,387956.25,2846.69,554
13,14,4353.0,407.0,438.0,424.0,401394.0,76264.86,325129.14,2385.68,525
14,15,8909.0,419.0,422.0,449.0,390002.0,81900.42,308101.58,2260.74,569
...,...,...,...,...,...,...,...,...,...,...
9983,9984,4705.0,375.0,403.0,208.0,290088.0,14504.40,275583.60,1833.46,633
9987,9988,9150.0,322.0,367.0,379.0,337324.0,87704.24,249619.76,1831.62,580
9994,9995,5374.0,349.0,376.0,401.0,234115.0,58528.75,175586.25,1288.39,599
9997,9998,9086.0,284.0,352.0,288.0,284712.0,56942.40,227769.60,1671.29,569


In [330]:
credit_score_status=[]
for i in list(df['CreditScore']):
    if i <=640:
        credit_score_status.append('N')
    else:
        credit_score_status.append('Y')
df['status_credit_score']=credit_score_status

In [331]:
df['status_credit_score'].value_counts()

Y    5935
N    4065
Name: status_credit_score, dtype: int64

In [332]:
loan_to_value=[]
df['Loan_to_value']=df['LoanAmount']/df['AppraisedValue']
for i in list(df['Loan_to_value']):
    if i <0.80:
        loan_to_value.append('Y')
    elif i>=0.95:
        loan_to_value.append('N')
    else:
        loan_to_value.append('Needs PMI')
df['Status_Loan_to_value']=loan_to_value

In [333]:
df['Status_Loan_to_value'].value_counts()

Needs PMI    5852
Y            3742
N             406
Name: Status_Loan_to_value, dtype: int64

In [334]:
df.columns

Index(['ID', 'GrossMonthlyIncome', 'CreditCardPayment', 'CarPayment',
       'StudentLoanPayments', 'AppraisedValue', 'DownPayment', 'LoanAmount',
       'MonthlyMortgagePayment', 'CreditScore', 'status_credit_score',
       'Loan_to_value', 'Status_Loan_to_value'],
      dtype='object')

In [335]:
df['Debt_to_income_ratio']=(df['CreditCardPayment'] + df['CarPayment']+ df['StudentLoanPayments'] + df['MonthlyMortgagePayment'])/(df['GrossMonthlyIncome'])

In [336]:
df['Debt_to_income_ratio']

0       0.666742
1       1.099354
2       0.688546
3       0.294183
4       1.792764
          ...   
9995    0.641190
9996    0.390598
9997    0.285636
9998    0.370250
9999    0.800362
Name: Debt_to_income_ratio, Length: 10000, dtype: float64

In [337]:
debt_to_income_status=[]

for i in list(df['Debt_to_income_ratio']):
    if i <0.36:
        debt_to_income_status.append('Y')
    elif ((i>=0.36) & (i<=0.43)):
        debt_to_income_status.append('Maybe')
    else:
        debt_to_income_status.append('N')
df['Status_Debt_to_income']=debt_to_income_status

In [338]:
df[['Debt_to_income_ratio','Status_Debt_to_income']].head(40)


,Debt_to_income_ratio,Status_Debt_to_income
0,0.666742,N
1,1.099354,N
2,0.688546,N
3,0.294183,Y
4,1.792764,N
5,0.282912,Y
6,0.256936,Y
7,0.278479,Y
8,0.252315,Y
9,0.438257,N


In [339]:
mmp_by_gmi=[]
df['MMP_by_GMI']=df['MonthlyMortgagePayment']/df['GrossMonthlyIncome']

for i in list(df['MMP_by_GMI']):
    if i <=0.28:
        mmp_by_gmi.append('Y')
    else:
        mmp_by_gmi.append('N')
df['Status_mmp_by_gmi']=mmp_by_gmi

In [340]:
df.columns
df['CreditCardPayment'][1]

368.0

In [341]:
df['status_credit_score'].iloc[3]

'Y'

In [342]:
df

,ID,GrossMonthlyIncome,CreditCardPayment,CarPayment,StudentLoanPayments,AppraisedValue,DownPayment,LoanAmount,MonthlyMortgagePayment,CreditScore,status_credit_score,Loan_to_value,Status_Loan_to_value,Debt_to_income_ratio,Status_Debt_to_income,MMP_by_GMI,Status_mmp_by_gmi
0,1,3103.0,317.0,374.0,250.0,268468.0,32216.16,236251.84,1127.90,778,Y,0.88,Needs PMI,0.666742,N,0.363487,N
1,2,2939.0,368.0,395.0,228.0,335467.0,30192.03,305274.97,2240.00,546,N,0.91,Needs PMI,1.099354,N,0.762164,N
2,3,4676.0,222.0,352.0,368.0,461177.0,36894.16,424282.84,2277.64,736,Y,0.92,Needs PMI,0.688546,N,0.487092,N
3,4,8469.0,224.0,416.0,400.0,326904.0,22883.28,304020.72,1451.44,761,Y,0.93,Needs PMI,0.294183,Y,0.171383,Y
4,5,2406.0,497.0,385.0,444.0,472661.0,23633.05,449027.95,2987.39,615,N,0.95,N,1.792764,N,1.241642,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,4329.0,483.0,473.0,306.0,440365.0,123302.20,317062.80,1513.71,754,Y,0.72,Y,0.641190,N,0.349667,N
9996,9997,7037.0,465.0,408.0,338.0,312762.0,56297.16,256464.84,1537.64,656,Y,0.82,Needs PMI,0.390598,Maybe,0.218508,Y
9997,9998,9086.0,284.0,352.0,288.0,284712.0,56942.40,227769.60,1671.29,569,N,0.80,Needs PMI,0.285636,Y,0.183941,Y
9998,9999,9655.0,443.0,365.0,407.0,387465.0,65869.05,321595.95,2359.76,700,Y,0.83,Needs PMI,0.370250,Maybe,0.244408,Y


In [343]:
for i in range (0, len(df)):
    if df['Status_Debt_to_income'].iloc[i]=='Maybe':
        if ((df['CreditScore'].iloc[i]>740) & (df['Loan_to_value'].iloc[i]>0.86)):
            df['Status_Debt_to_income'].iloc[i]='Y'
        else:
            df['Status_Debt_to_income'].iloc[i]='N'

/var/folders/cy/d96s2_cs0ms966dbvj6lm2qr0000gn/T/ipykernel_65221/1733965585.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Status_Debt_to_income'].iloc[i]='N'
/var/folders/cy/d96s2_cs0ms966dbvj6lm2qr0000gn/T/ipykernel_65221/1733965585.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Status_Debt_to_income'].iloc[i]='Y'


In [344]:
for i in range (0, len(df)):
    if df['Status_Loan_to_value'].iloc[i]=='Needs PMI':
        if ((df['CreditScore'].iloc[i]>740) & (df['Debt_to_income_ratio'].iloc[i]<=0.385)):
            df['Status_Loan_to_value'].iloc[i]='Y'
        else:
            df['Status_Loan_to_value'].iloc[i]='N'

/var/folders/cy/d96s2_cs0ms966dbvj6lm2qr0000gn/T/ipykernel_65221/197750408.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Status_Loan_to_value'].iloc[i]='N'
/var/folders/cy/d96s2_cs0ms966dbvj6lm2qr0000gn/T/ipykernel_65221/197750408.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Status_Loan_to_value'].iloc[i]='Y'


In [345]:
df['Status_Loan_to_value'].value_counts()

N    5486
Y    4514
Name: Status_Loan_to_value, dtype: int64

In [346]:
# df_for_dti_maybe_clustering=df.query('Status_Debt_to_income=="Maybe" & status_credit_score=="Y" & Status_mmp_by_gmi=="Y" & Status_Loan_to_value!="N"')

In [347]:
# df_for_dti_maybe_clustering.columns

In [348]:
# x_for_clustering=df_for_dti_maybe_clustering[['CreditScore', 'Debt_to_income_ratio','Loan_to_value']]

In [349]:
# x_for_clustering

In [350]:
# x_for_clustering = StandardScaler().fit_transform(x_for_clustering)

In [351]:
# kmeans = KMeans(2)
# kmeans.fit(x_for_clustering)

In [352]:
# identified_clusters = kmeans.fit_predict(x_for_clustering)
# identified_clusters

In [353]:
# data_with_clusters = df_for_dti_maybe_clustering.copy()
# data_with_clusters['Clusters'] = identified_clusters 
# plt.scatter(data_with_clusters['CreditScore'],data_with_clusters['Debt_to_income_ratio'],c=data_with_clusters['Clusters'],cmap='rainbow')

In [354]:
# start_subs_dti = 0
# start_subs_pmi = 0

# for i in range(len(list(df['Status_Debt_to_income']))):
#     flag1 = -1
#     if((df['Status_Debt_to_income'][i]=="Maybe") and (df['status_credit_score'][i]=="Y") and (df['Status_mmp_by_gmi'][i]=="Y") and (df['Status_Loan_to_value'][i]!="N")):
#         if(identified_clusters[start_subs_dti]==0):
#             flag1 = 0
#         else:
#             flag1 = 1
#         start_subs_dti+=1
#     if(flag1==0):
#         df['Status_Debt_to_income'].iloc[i]='N'
#     if(flag1==1):
#         df['Status_Debt_to_income'].iloc[i]='Y'

In [355]:
# wcss=[]
# for i in range(1,15):
#     kmeans = KMeans(i)
#     kmeans.fit(x_for_clustering)
#     wcss_iter = kmeans.inertia_
#     wcss.append(wcss_iter)

# number_clusters = range(1,15)
# plt.plot(number_clusters,wcss)
# plt.title('The Elbow title')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')

In [356]:
# df_for_ltv_pmi_clustering=df.query('Status_Loan_to_value=="Needs PMI" & status_credit_score=="Y" & Status_mmp_by_gmi=="Y" & Status_Debt_to_income != "N"')

In [357]:
# x_pmi_for_clustering=df_for_ltv_pmi_clustering[['CreditScore', 'Debt_to_income_ratio','Loan_to_value']]

In [358]:
# x_pmi_for_clustering

In [359]:
# x_pmi_for_clustering = StandardScaler().fit_transform(x_pmi_for_clustering)
# kmeans_pmi = KMeans(2)
# kmeans_pmi.fit(x_pmi_for_clustering)

In [360]:
# identified_clusters_pmi = kmeans_pmi.fit_predict(x_pmi_for_clustering)
# identified_clusters_pmi

In [361]:
# data_with_clusters = df_for_ltv_pmi_clustering.copy()
# data_with_clusters['Clusters'] = identified_clusters_pmi 
# plt.scatter(data_with_clusters['Debt_to_income_ratio'],data_with_clusters['Loan_to_value'],c=data_with_clusters['Clusters'],cmap='rainbow')

In [362]:
# wcss=[]
# for i in range(1,7):
#     kmeans_pmi = KMeans(i)
#     kmeans_pmi.fit(x_pmi_for_clustering)
#     wcss_iter = kmeans.inertia_
#     wcss.append(wcss_iter)

# number_clusters = range(1,7)
# plt.plot(number_clusters,wcss)
# plt.title('The Elbow title')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')

In [363]:
len(identified_clusters_pmi)

1333

In [364]:
identified_clusters_pmi

array([1, 1, 0, ..., 0, 0, 0], dtype=int32)

In [365]:

# start_subs_pmi = 0

# for i in range(len(list(df['Status_Debt_to_income']))):
#     flag2 = -1
#     if((df['Status_Debt_to_income'][i]!="N") and (df['status_credit_score'][i]=="Y") and (df['Status_mmp_by_gmi'][i]=="Y") and (df['Status_Loan_to_value'][i]=="Needs PMI")):
#         if(identified_clusters_pmi[start_subs_pmi]==0):
#             flag2=0
#         else:
#             flag2=1
#         start_subs_pmi+=1
#     if(flag2==0):
#         df['Status_Loan_to_value'].iloc[i]='N'
#     if(flag2==1):
#         df['Status_Loan_to_value'].iloc[i]=='Y'

In [366]:
# final_status=[]
# for i in range (0, len(df)):
#     if ((df['status_credit_score'].iloc[i]=='N')or(df['Status_Debt_to_income'].iloc[i]=='N')or(df['Status_Loan_to_value'].iloc[i]=='N')or (df['Status_mmp_by_gmi'].iloc[i]=='N')):
#         final_status.append('N')
#     else:
#         final_status.append('Y')


In [367]:
# final_status=[]
# for i in range (0, len(df)):
#     if ((df['status_credit_score'].iloc[i]=='N') or (df['Status_mmp_by_gmi'].iloc[i]=='N') or (df['Status_Debt_to_income'].iloc[i]=='N') or (df['Status_Loan_to_value'].iloc[i]=='N')):
#         final_status='N'
#     else:
#         final_status='Y'
# df['final_status']=final_status
    

In [368]:
# df['final_status'].value_counts()

In [369]:
df['Status_Loan_to_value'].value_counts()

N    5486
Y    4514
Name: Status_Loan_to_value, dtype: int64

In [370]:
df['Status_Debt_to_income'].value_counts()

N    7291
Y    2709
Name: Status_Debt_to_income, dtype: int64

In [371]:
start_subs_dti

740

In [372]:
start_subs_pmi

1333

In [373]:

# df[(df['Status_Debt_to_income']!="N") & (df['status_credit_score']=="Y") & (df['Status_mmp_by_gmi']=="Y") & (df['Status_Loan_to_value']=="Needs PMI")]

In [375]:
df['Status_Debt_to_income'].value_counts()

N    7291
Y    2709
Name: Status_Debt_to_income, dtype: int64

In [377]:
df['Status_Loan_to_value'].value_counts()

N    5486
Y    4514
Name: Status_Loan_to_value, dtype: int64

In [378]:
df['Status_mmp_by_gmi'].value_counts()

N    5586
Y    4414
Name: Status_mmp_by_gmi, dtype: int64

In [379]:
df['status_credit_score'].value_counts()

Y    5935
N    4065
Name: status_credit_score, dtype: int64

In [386]:
df[df['status_credit_score']=='Y']

,ID,GrossMonthlyIncome,CreditCardPayment,CarPayment,StudentLoanPayments,AppraisedValue,DownPayment,LoanAmount,MonthlyMortgagePayment,CreditScore,status_credit_score,Loan_to_value,Status_Loan_to_value,Debt_to_income_ratio,Status_Debt_to_income,MMP_by_GMI,Status_mmp_by_gmi
0,1,3103.0,317.0,374.0,250.0,268468.0,32216.16,236251.84,1127.90,778,Y,0.88,N,0.666742,N,0.363487,N
2,3,4676.0,222.0,352.0,368.0,461177.0,36894.16,424282.84,2277.64,736,Y,0.92,N,0.688546,N,0.487092,N
3,4,8469.0,224.0,416.0,400.0,326904.0,22883.28,304020.72,1451.44,761,Y,0.93,Y,0.294183,Y,0.171383,Y
5,6,9128.0,219.0,424.0,220.0,403922.0,117137.38,286784.62,1719.42,661,Y,0.71,Y,0.282912,Y,0.188368,Y
6,7,8717.0,249.0,363.0,395.0,365483.0,73096.60,292386.40,1232.71,811,Y,0.80,Y,0.256936,Y,0.141414,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,9993,2010.0,261.0,450.0,278.0,397523.0,31801.84,365721.16,1963.27,721,Y,0.92,N,1.468791,N,0.976751,N
9993,9994,6399.0,249.0,358.0,202.0,310916.0,77729.00,233187.00,1398.07,697,Y,0.75,Y,0.344909,Y,0.218483,Y
9995,9996,4329.0,483.0,473.0,306.0,440365.0,123302.20,317062.80,1513.71,754,Y,0.72,Y,0.641190,N,0.349667,N
9996,9997,7037.0,465.0,408.0,338.0,312762.0,56297.16,256464.84,1537.64,656,Y,0.82,N,0.390598,N,0.218508,Y


In [387]:
yes_df=df[(df['Status_Debt_to_income']=='Y') & (df['Status_Loan_to_value']=='Y') & (df['Status_mmp_by_gmi']=='Y') & (df['status_credit_score']=='Y')]

,ID,GrossMonthlyIncome,CreditCardPayment,CarPayment,StudentLoanPayments,AppraisedValue,DownPayment,LoanAmount,MonthlyMortgagePayment,CreditScore,status_credit_score,Loan_to_value,Status_Loan_to_value,Debt_to_income_ratio,Status_Debt_to_income,MMP_by_GMI,Status_mmp_by_gmi
3,4,8469.0,224.0,416.0,400.0,326904.0,22883.28,304020.72,1451.44,761,Y,0.93,Y,0.294183,Y,0.171383,Y
5,6,9128.0,219.0,424.0,220.0,403922.0,117137.38,286784.62,1719.42,661,Y,0.71,Y,0.282912,Y,0.188368,Y
6,7,8717.0,249.0,363.0,395.0,365483.0,73096.60,292386.40,1232.71,811,Y,0.80,Y,0.256936,Y,0.141414,Y
7,8,8194.0,315.0,353.0,303.0,269907.0,72874.89,197032.11,1310.86,648,Y,0.73,Y,0.278479,Y,0.159978,Y
29,30,6461.0,295.0,381.0,395.0,239093.0,57382.32,181710.68,1089.45,666,Y,0.76,Y,0.334383,Y,0.168619,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9965,9966,9033.0,364.0,351.0,347.0,261617.0,41858.72,219758.28,1049.16,763,Y,0.84,Y,0.233716,Y,0.116147,Y
9967,9968,9684.0,392.0,389.0,232.0,294405.0,32384.55,262020.45,1104.69,847,Y,0.89,Y,0.218679,Y,0.114074,Y
9968,9969,8666.0,338.0,418.0,257.0,361264.0,25288.48,335975.52,1604.00,765,Y,0.93,Y,0.301985,Y,0.185091,Y
9976,9977,6605.0,340.0,397.0,271.0,334544.0,30108.96,304435.04,1453.42,751,Y,0.91,Y,0.372660,Y,0.220048,Y


In [395]:
aggregated_status=[]
for i in range (len(df)):
    if ((df['Status_Debt_to_income'].iloc[i]=='Y') & (df['Status_Loan_to_value'].iloc[i]=='Y') & (df['Status_mmp_by_gmi'].iloc[i]=='Y') & (df['status_credit_score'].iloc[i]=='Y')):
        aggregated_status.append('Y')
    else:
        aggregated_status.append('N')

In [397]:
df['Final_status']=aggregated_status

In [398]:
df['Final_status'].value_counts()

N    8444
Y    1556
Name: Final_status, dtype: int64

In [399]:
df.columns

Index(['ID', 'GrossMonthlyIncome', 'CreditCardPayment', 'CarPayment',
       'StudentLoanPayments', 'AppraisedValue', 'DownPayment', 'LoanAmount',
       'MonthlyMortgagePayment', 'CreditScore', 'status_credit_score',
       'Loan_to_value', 'Status_Loan_to_value', 'Debt_to_income_ratio',
       'Status_Debt_to_income', 'MMP_by_GMI', 'Status_mmp_by_gmi',
       'Final_status'],
      dtype='object')

In [429]:
one_hot_encoded_data = pd.get_dummies(df, columns = ['status_credit_score', 'Status_Loan_to_value','Status_Debt_to_income','Status_mmp_by_gmi'])

In [419]:
one_hot_encoded_data.data=one_hot_encoded_data[['GrossMonthlyIncome', 'CreditCardPayment', 'CarPayment',
       'StudentLoanPayments', 'AppraisedValue', 'DownPayment', 'LoanAmount',
       'MonthlyMortgagePayment', 'CreditScore', 'Loan_to_value',
       'Debt_to_income_ratio', 'MMP_by_GMI', 
       'status_credit_score_N', 'status_credit_score_Y',
       'Status_Loan_to_value_N', 'Status_Loan_to_value_Y',
       'Status_Debt_to_income_N', 'Status_Debt_to_income_Y',
       'Status_mmp_by_gmi_N', 'Status_mmp_by_gmi_Y']]

/var/folders/cy/d96s2_cs0ms966dbvj6lm2qr0000gn/T/ipykernel_65221/4062249291.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  one_hot_encoded_data.data=one_hot_encoded_data[['GrossMonthlyIncome', 'CreditCardPayment', 'CarPayment',


In [417]:
one_hot_encoded_data.columns

Index(['ID', 'GrossMonthlyIncome', 'CreditCardPayment', 'CarPayment',
       'StudentLoanPayments', 'AppraisedValue', 'DownPayment', 'LoanAmount',
       'MonthlyMortgagePayment', 'CreditScore', 'Loan_to_value',
       'Debt_to_income_ratio', 'MMP_by_GMI', 'Final_status',
       'status_credit_score_N', 'status_credit_score_Y',
       'Status_Loan_to_value_N', 'Status_Loan_to_value_Y',
       'Status_Debt_to_income_N', 'Status_Debt_to_income_Y',
       'Status_mmp_by_gmi_N', 'Status_mmp_by_gmi_Y'],
      dtype='object')

In [420]:
one_hot_encoded_data.target=one_hot_encoded_data['Final_status']

/var/folders/cy/d96s2_cs0ms966dbvj6lm2qr0000gn/T/ipykernel_65221/2314028213.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  one_hot_encoded_data.target=one_hot_encoded_data['Final_status']


In [424]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(one_hot_encoded_data.data, one_hot_encoded_data.target, test_size=0.3,random_state=109) # 70% training and 30% test

In [425]:
y_test

6041    N
1677    N
4352    Y
5212    N
1691    N
       ..
7422    N
2489    N
1941    N
8230    N
921     N
Name: Final_status, Length: 3000, dtype: object

In [426]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [427]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.948


In [413]:
df.columns

Index(['ID', 'GrossMonthlyIncome', 'CreditCardPayment', 'CarPayment',
       'StudentLoanPayments', 'AppraisedValue', 'DownPayment', 'LoanAmount',
       'MonthlyMortgagePayment', 'CreditScore', 'status_credit_score',
       'Loan_to_value', 'Status_Loan_to_value', 'Debt_to_income_ratio',
       'Status_Debt_to_income', 'MMP_by_GMI', 'Status_mmp_by_gmi',
       'Final_status'],
      dtype='object')